In [1]:
# preliminary data preprocessing

In [2]:
# import file

import csv

file_path = "datasets\\data1.csv"

with open(file_path, 'r') as csv_file:
    reader = csv.reader(csv_file)

In [3]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow.python

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow.python (from versions: none)
ERROR: No matching distribution found for tensorflow.python


In [5]:
# import packages

import datetime
import os
import collections

import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from matplotlib import pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import (accuracy_score, auc, classification_report,
                             confusion_matrix, f1_score, precision_recall_curve,
                             precision_score, recall_score, roc_auc_score,
                             roc_curve)
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import (cross_validate, train_test_split)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import KFold

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [6]:
dataset_file = file_path
df = pd.read_csv(dataset_file)
df.shape

(452, 15)

In [7]:
# display file as dataframe

pd.set_option('display.max_columns', None)
df

,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,2021-11-03 02:30:00,2021-11-03 08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,2021-03-13 01:00:00,2021-03-13 09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,448,27,Female,2021-11-13 22:00:00,2021-11-13 05:30:00,7.5,0.91,22,57,21,0.0,0.0,0.0,No,5.0
448,449,52,Male,2021-03-31 21:00:00,2021-03-31 03:00:00,6.0,0.74,28,57,15,4.0,25.0,0.0,No,3.0
449,450,40,Female,2021-09-07 23:00:00,2021-09-07 07:30:00,8.5,0.55,20,32,48,1.0,NaN,3.0,Yes,0.0
450,451,45,Male,2021-07-29 21:00:00,2021-07-29 04:00:00,7.0,0.76,18,72,10,3.0,0.0,0.0,No,3.0


In [8]:
df.columns

Index(['ID', 'Age', 'Gender', 'Bedtime', 'Wakeup time', 'Sleep duration',
       'Sleep efficiency', 'REM sleep percentage', 'Deep sleep percentage',
       'Light sleep percentage', 'Awakenings', 'Caffeine consumption',
       'Alcohol consumption', 'Smoking status', 'Exercise frequency'],
      dtype='object')

In [9]:
df.dtypes

ID                          int64
Age                         int64
Gender                     object
Bedtime                    object
Wakeup time                object
Sleep duration            float64
Sleep efficiency          float64
REM sleep percentage        int64
Deep sleep percentage       int64
Light sleep percentage      int64
Awakenings                float64
Caffeine consumption      float64
Alcohol consumption       float64
Smoking status             object
Exercise frequency        float64
dtype: object

In [10]:
# data preprocessing

df.columns = df.columns.str.strip()

In [11]:
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})
df['Smoking status'] = df['Smoking status'].map({'No': 0, 'Yes': 1})


df['Awakenings'].fillna(0, inplace = True)
df['Alcohol consumption'].fillna(0, inplace = True)
df['Caffeine consumption'].fillna(0, inplace = True)
df['Exercise frequency'].fillna(0, inplace = True)

In [12]:
df = df.loc[:,~df.columns.duplicated()]

# reorder columns for dataframe

new_order = [
    'Age',
    'Gender',
    'Sleep duration',
    'Sleep efficiency',
    'REM sleep percentage',
    'Deep sleep percentage',
    'Light sleep percentage',
    'Awakenings',
    'Caffeine consumption',
    'Alcohol consumption',
    'Smoking status',
    'Exercise frequency',
]

df = df.reindex(columns = new_order)


In [13]:
df

,Age,Gender,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,65,1,6.0,0.88,18,70,12,0.0,0.0,0.0,1,3.0
1,69,0,7.0,0.66,19,28,53,3.0,0.0,3.0,1,3.0
2,40,1,8.0,0.89,20,70,10,1.0,0.0,0.0,0,3.0
3,40,1,6.0,0.51,23,25,52,3.0,50.0,5.0,1,1.0
4,57,0,8.0,0.76,27,55,18,3.0,0.0,3.0,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
447,27,1,7.5,0.91,22,57,21,0.0,0.0,0.0,0,5.0
448,52,0,6.0,0.74,28,57,15,4.0,25.0,0.0,0,3.0
449,40,1,8.5,0.55,20,32,48,1.0,0.0,3.0,1,0.0
450,45,0,7.0,0.76,18,72,10,3.0,0.0,0.0,0,3.0


In [14]:
df.dtypes

Age                         int64
Gender                      int64
Sleep duration            float64
Sleep efficiency          float64
REM sleep percentage        int64
Deep sleep percentage       int64
Light sleep percentage      int64
Awakenings                float64
Caffeine consumption      float64
Alcohol consumption       float64
Smoking status              int64
Exercise frequency        float64
dtype: object

In [15]:
df.to_csv('datasets\\data2.csv')